# kNN & Linear Regression From Scratch: Iris Dataset
> A somewhat comprehensive record of the steps I took to set up this blog; upgrades I made to the original fastpages setup; issues I faced and how I resolved them (or not); and references to resources I found useful.

- toc: true
- author: Oluwaleke Umar Yusuf
- badges: true
- comments: true
- image: images/ipynb/iris_dataset.png
- hide: false
- search_exclude: false
- categories: [notebook, fastpages]
- tags: [fastpages examples]